In [1]:
import numpy as np
import heapq

## Part 1

In [ ]:
directions = ['^', '>', 'v', '<']
delta = {'^': (-1, 0), '>': (0, 1), 'v': (1, 0), '<': (0, -1)}

def read_text_file_as_grid(file_path):
    with open(file_path, 'r') as file:
        grid = [list(line.strip()) for line in file if line.strip()]
    return np.array(grid)

def is_inside_grid(grid, x, y):
    rows, cols = grid.shape
    return 0 <= x < rows and 0 <= y < cols

def is_valid_move(grid, x, y):
    return is_inside_grid(grid, x, y) and grid[x][y] != '#'

def find_start_and_end(grid):
    start = tuple(np.argwhere(grid=="S")[0])
    end = tuple(np.argwhere(grid=="E")[0])
    return start, end

def bfs_with_cost(grid):
    start, end = find_start_and_end(grid)
    start_x, start_y = start
    goal_x, goal_y = end
    
    queue = [(0, start_x, start_y, '>')] 
    heapq.heapify(queue)
    
    visited = set()
    
    parent = {}
    
    while queue:
        cost, x, y, direction = heapq.heappop(queue)
        
        if (x, y) == (goal_x, goal_y):
            path, tiles = reconstruct_path(parent, start_x, start_y, goal_x, goal_y, direction)
            return  path, tiles, cost
        
        if (x, y, direction) in visited:
            continue
        
        visited.add((x, y, direction))
        
        dx, dy = delta[direction]
        nx, ny = x + dx, y + dy
        if is_valid_move(grid, nx, ny) and (nx, ny, direction) not in visited:
            heapq.heappush(queue, (cost + 1, nx, ny, direction))
            parent[(nx, ny, direction)] = (x, y, direction)
        
        left_direction = directions[(directions.index(direction) - 1) % 4]
        if (x, y, left_direction) not in visited:
            heapq.heappush(queue, (cost + 1000, x, y, left_direction))
            parent[(x, y, left_direction)] = (x, y, direction)
        
        right_direction = directions[(directions.index(direction) + 1) % 4]
        if (x, y, right_direction) not in visited:
            heapq.heappush(queue, (cost + 1000, x, y, right_direction))
            parent[(x, y, right_direction)] = (x, y, direction)

    return None, None, float('inf')

def reconstruct_path(parent, start_x, start_y, goal_x, goal_y, final_direction):
    path = []
    current = (goal_x, goal_y, final_direction)
    count_tiles = 0
    
    while current in parent:
        x, y, direction = current
        path.append((x, y, direction))
        if grid[x][y] not in ["S","E","#"]:
            grid[x][y] = direction
            count_tiles += 1
        current = parent[current]
    
    path.append((start_x, start_y, '>'))  
    path.reverse()
    return path, count_tiles + 2

input_file = "example1.txt"
grid = read_text_file_as_grid(input_file)
path, count_tiles, total_cost = bfs_with_cost(grid)

if path:
    print(f"total cost: {total_cost}")
    for row in grid:
        print(' '.join(row))


total cost: 7036
# # # # # # # # # # # # # # #
# . . . . . . . # . . . . E #
# . # . # # # . # . # # # ^ #
# . . . . . # . # . . . # ^ #
# . # # # . # # # # # . # ^ #
# . # . # . . . . . . . # ^ #
# . # . # # # # # . # # # ^ #
# . . ^ > > > > > > > > # ^ #
# # # ^ # . # # # # # v # ^ #
# . . ^ # . . . . . # v # ^ #
# . # ^ # . # # # . # v # ^ #
# ^ > > . . # . . . # v # ^ #
# ^ # # # . # . # . # v # ^ #
# S . . # . . . . . # v > > #
# # # # # # # # # # # # # # #


In [10]:
input_file = "input.txt"
grid = read_text_file_as_grid(input_file)
path, tiles, total_cost = bfs_with_cost(grid)

print(f"total cost: {total_cost}")

total cost: 91464


## Part 2

In [ ]:
def reconstruct_paths(parents, start, goal):
    paths = []
    stack = [(goal, [])]

    while stack:
        current, path = stack.pop()

        if (current[0], current[1]) == start:
            paths.append([(start[0], start[1], '>')] + path[::-1])
            continue

        if current not in parents:
            continue

        for parent in parents[current]:
            stack.append((parent, path + [current]))

    return paths

def bfs_find_all_optimal_paths(grid):
    start, end = find_start_and_end(grid)
    start_x, start_y = start
    goal_x, goal_y = end

    queue = [(0, start_x, start_y, '>')] 
    heapq.heapify(queue)
    
    cost_so_far = {}

    parents = {}

    optimal_goal_states = []
    optimal_cost = float('inf')

    while queue:
        cost, x, y, direction = heapq.heappop(queue)

        if cost > optimal_cost:
            continue
        
        if (x, y) == (goal_x, goal_y):
            if cost < optimal_cost:
                optimal_cost = cost
                optimal_goal_states = [(x, y, direction)]
            elif cost == optimal_cost:
                optimal_goal_states.append((x, y, direction))
            continue

        if (x, y, direction) in cost_so_far and cost_so_far[(x, y, direction)] <= cost:
            continue

        cost_so_far[(x, y, direction)] = cost

        dx, dy = delta[direction]
        nx, ny = x + dx, y + dy
        if is_valid_move(grid, nx, ny):
            new_cost = cost + 1
            if (nx, ny, direction) not in cost_so_far or cost_so_far[(nx, ny, direction)] > new_cost:
                heapq.heappush(queue, (new_cost, nx, ny, direction))
                parents.setdefault((nx, ny, direction), []).append((x, y, direction))

        left_direction = directions[(directions.index(direction) - 1) % 4]
        new_cost = cost + 1000
        if (x, y, left_direction) not in cost_so_far or cost_so_far[(x, y, left_direction)] > new_cost:
            heapq.heappush(queue, (new_cost, x, y, left_direction))
            parents.setdefault((x, y, left_direction), []).append((x, y, direction))

        right_direction = directions[(directions.index(direction) + 1) % 4]
        new_cost = cost + 1000
        if (x, y, right_direction) not in cost_so_far or cost_so_far[(x, y, right_direction)] > new_cost:
            heapq.heappush(queue, (new_cost, x, y, right_direction))
            parents.setdefault((x, y, right_direction), []).append((x, y, direction))

    all_paths = []
    for goal in optimal_goal_states:
        all_paths.extend(reconstruct_paths(parents, start, goal))
    
    return all_paths, optimal_cost

In [37]:
input_file = "input.txt"
grid = read_text_file_as_grid(input_file)
all_paths, total_cost = bfs_find_all_optimal_paths(grid)

In [38]:
tiles = []
for path in all_paths:
    for step in path:
        if (step[0], step[1]) not in tiles:
            tiles.append((step[0], step[1]))
            
print(len(tiles))

494
